In [14]:
#7: Use BigQuery public data for names in USA (`bigquery-public-data.usa_names.usa_1910_current`). 
# Write a query to find the most common male and female name for every year. 
# Save results to table US_common_names. Use BigQuery API and action query in a way that the 
# data is not moved from BigQuery.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import os

In [16]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/tuukka/gcp_keys/atlantean-stock-385411-fce448540d0c.json"
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

/home/tuukka/gcp_keys/atlantean-stock-385411-fce448540d0c.json


In [17]:
projectid = "atlantean-stock-385411"

sql = """
SELECT
year,
gender,
name,
number
FROM (
SELECT
  name,
  gender,
  year,
  number,
  RANK() OVER (
    PARTITION BY gender, year
    ORDER BY number DESC
  ) AS rn
FROM `bigquery-public-data.usa_names.usa_1910_current`
)
WHERE rn = 1
ORDER BY year, gender
"""

from google.cloud import bigquery
client = bigquery.Client()

#define the data destination, sql defines the source
projectid = "atlantean-stock-385411"
dataset = 'data_engineering'
table_name = 'us_names'

destination = projectid + '.' + dataset + '.' + table_name

job_config = bigquery.QueryJobConfig(destination = destination)
job_config.write_disposition = 'WRITE_TRUNCATE'

query_job = client.query(sql,job_config = job_config)
query_job.result()
print("Query results loaded to the table {}".format(destination))


Query results loaded to the table atlantean-stock-385411.data_engineering.us_names
